# Installation

In [1]:
#!pip install folium
#!pip install altair

# Imports

In [70]:
import pandas as pd
import folium
from folium.features import FeatureGroup, VegaLite
import altair as alt
from branca.element import *

# Example with Circle Markers and Popups with Bar Charts

In [71]:
# Read 'xlsx' file as pandas dataframe
df = pd.read_excel(r'data/hospitales.xlsx', converters={'Long':str, 'Lat':str}, index_col=None)
df_tr = pd.read_excel(r'data/hospitales_process.xlsx', converters={'Long':str, 'Lat':str}, index_col=None)

# Remove rows with value '0' on column 'Long'
df = df.drop(df['Long'].loc[df['Long']=="0"].index)
# Create a list with the index of df
count_hospitals = list(df.index)

# Create a map
m = folium.Map(location=[22.037635528256533, -79.36879425670136], zoom_start=7, 
               tiles=None)

# Add a title
loc = 'Hospitales con denuncias de maltrato en Cuba'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)  
m.get_root().html.add_child(folium.Element(title_html))

# Add layer and set dark as default mode
folium.TileLayer(tiles="CartoDB dark_matter", name="Dark", attr= '''
               &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
               contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
               ''').add_to(m)
# Add layer for light mode
folium.TileLayer(tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png', 
                 name="Light", attr= '''
                 &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
                 contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
                ''', overlay=False).add_to(m)

# Add the option to switch layers
hospitales = FeatureGroup(name="Hospitales")
hospitales.add_to(m)
folium.LayerControl('topright', collapsed= False).add_to(m)

# List with columns names
columns_list = list(df_tr)

# Empty list for charts
charts = []
# Create a chart for each hospital column
for i in range(len(columns_list)):
    charts.append("chart"+'i')
    charts[i] = alt.Chart(df_tr).mark_bar(color='blue', size=20).encode(
            alt.X(columns_list[i+2], title='Hospital', 
                  scale=alt.Scale(domain=[0, 30])),
            alt.Y(columns_list[0], title='Tipos', 
                  sort=alt.EncodingSortField(field=columns_list[i+2], op="count", order='ascending')),
            tooltip=columns_list[1]).interactive().properties(title='Tipos de maltrato por Hospital', 
                                                              width=350, height=300)
    # Save chart as json
    charts[i].to_json()
    if i == len(count_hospitals)-1:
        break

# Add circle markers for each row in df
k = 0
for j, r in df.iterrows():
    folium.CircleMarker([r['Long'], r['Lat']],
    radius=r['Total']/3,
    color='#9b00ff',
    fill_color='#9b00ff',
    fill_opacity=(0.5),
    tooltip=r['Hospital'],
    # Add popup with chart
    popup=folium.Popup(max_width=450, max_heigth=400).add_child(folium.VegaLite(charts[k], width=450, height=300))
    ).add_to(hospitales)
    k += 1

# Save map as 'html'
#m.save("index.html")
    
# Show map
m

# Testing